In [ ]:
%pip install azure-search-documents==11.6.0b12

### REFERENCE 
https://techcommunity.microsoft.com/blog/azure-ai-services-blog/introducing-agentic-retrieval-in-azure-ai-search/4414677

In [13]:
import os
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchField,SearchFieldDataType,SimpleField,SearchableField

In [9]:
# Load environment variables

AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")  # e.g. https://<your-search>.search.windows.net
AZURE_SEARCH_KEY = os.getenv("AZURE_SEARCH_KEY")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")  # Your index name
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_EMBEDDING_MODEL = "text-embedding-3-small"
EMBEDDING_DIM = 1536  # dimension for OpenAI 'text-embedding-3-small'

AOAI_ENDPOINT = ''
AOAI_EMB_MODEL = ''
AOAI_EMB_DEPLOYMENT = ''
AOAI_GPT_MODEL = ''
AOAI_GPT_DEPLOYMENT = ''

In [10]:
# Define fields for the index
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SimpleField(name="page", type=SearchFieldDataType.Int32, filterable=True, sortable=True),
    SearchableField(name="content", type=SearchFieldDataType.String, analyzer_name="en.microsoft"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                hidden=False, searchable=True, vector_search_dimensions=EMBEDDING_DIM,
                vector_search_profile_name="default-vs")
]

In [ ]:
# Define vector search profile
from azure.search.documents.indexes.models import SearchIndex, SearchField, VectorSearch, VectorSearchProfile, HnswAlgorithmConfiguration, AzureOpenAIVectorizer, AzureOpenAIVectorizerParameters, SemanticSearch, SemanticConfiguration, SemanticPrioritizedFields, SemanticField

vector_search=VectorSearch(
    profiles=[VectorSearchProfile(name="hnsw_text_3_large", algorithm_configuration_name="alg", vectorizer_name="azure_openai_text_3_large")],
    algorithms=[HnswAlgorithmConfiguration(name="alg")],
    vectorizers=[
        AzureOpenAIVectorizer(
            vectorizer_name="azure_openai_text_3_large",
            parameters=AzureOpenAIVectorizerParameters(
                resource_url=AOAI_ENDPOINT,
                deployment_name=AOAI_EMB_DEPLOYMENT,
                model_name=AOAI_EMB_MODEL,
            )
        )
    ]
)


In [ ]:
semantic_search=SemanticSearch(
    default_configuration_name="semantic_config",
    configurations=[
        SemanticConfiguration(
            name="semantic_config",
            prioritized_fields=SemanticPrioritizedFields(
                content_fields=[
                    SemanticField(field_name="text")
                ]
            )
        )
    ]
)

In [ ]:
# Init Azure Search client
credential = AzureKeyCredential(AZURE_SEARCH_KEY)
index_client = SearchIndexClient(endpoint=AZURE_SEARCH_ENDPOINT, credential=credential)

# Create index
index = SearchIndex(
    name=AZURE_SEARCH_INDEX_NAME,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search
)

index_client.create_index(index)

print(f"Index '{AZURE_SEARCH_INDEX_NAME}' created.")

search_client = SearchClient(endpoint=AZURE_SEARCH_ENDPOINT, index_name=AZURE_SEARCH_INDEX_NAME, credential=credential)

In [ ]:
# Ingest documents

In [ ]:
# Create a agent in azure search 
from azure.search.documents.indexes.models import AzureOpenAIVectorizerParameters, KnowledgeAgent, KnowledgeAgentAzureOpenAIModel, KnowledgeAgentTargetIndex

agent_name = ''
agent = KnowledgeAgent(
    name=agent_name,
    models=[
        KnowledgeAgentAzureOpenAIModel(
            azure_open_ai_parameters=AzureOpenAIVectorizerParameters(
                resource_url=AOAI_ENDPOINT,
                deployment_name=AOAI_GPT_DEPLOYMENT,
                model_name=AOAI_GPT_MODEL)
        )
    ],
    target_indexes=[
        KnowledgeAgentTargetIndex(
            index_name=AZURE_SEARCH_INDEX_NAME,
            default_reranker_threshold=2.5,
            default_include_reference_source_data = True,
        )
    ],
)

index_client.create_or_update_agent(agent)
print(f"Knowledge agent -  '{agent_name}' created or updated successfully")

In [ ]:
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
agent_client = KnowledgeAgentRetrievalClient(endpoint=AZURE_SEARCH_ENDPOINT,
                    agent_name=agent_name,
                    credential=credential
)

In [ ]:
QUESTION = "user question"

instructions = """
You are a Q&A legal agent capable of answering questions related to a legal document.
The sources are provided in JSON format and include a "doc_key" reference that must be cited in the responses. 
If the answer is not available, the agent should reply with "I DO NOT KNOW".
"""

messages = [{"role": "system", "content": instructions},{ "role": "user", "content": QUESTION}]


In [ ]:
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, KnowledgeAgentIndexParams

retrieval_result = agent_client.retrieve(
    retrieval_request=KnowledgeAgentRetrievalRequest(
        messages=[
            KnowledgeAgentMessage(role=msg["role"],
                                  content=[
                                      KnowledgeAgentMessageTextContent(
                                          text=msg["content"])
                                  ]) for msg in messages
            if msg["role"] != "system"
        ],
        target_index_params=[
            KnowledgeAgentIndexParams(index_name=AZURE_SEARCH_INDEX_NAME,
                                      reranker_threshold=2.5)
]))
retrieval_result